In [1]:
# Initialization
a = .7
x = np.array([np.pi/2-a, 2*a, -a, 3*np.pi/4, 3*np.pi/4]) # Initial robot state

# Control loop definition
def control(x, param):
    # Coordinated control (position tracking with coordination matrix)
    u = np.zeros(param.nbVarX)
    # print(u)

    if move_joint >= 0:
        # Imposed coordination matrix (no correlations imposed on the last two joints)
        C = [[-1,0,0], [2,0,0], [-1,0,0], [0,1,0], [0,0,1]]
        # Residual and Jacobian
        df = (mouse - fkin(x[:move_joint+1], param2)) * 5  # mouse:鼠标点击位置
        print('mouse:', mouse)
        print('param2:', param2)
        print('fkin(x[:move_joint+1], param2):', fkin(x[:move_joint+1], param2))
        print('df:', df)
        J = Jkin(x[:move_joint+1], param2)
        print('J:', J)
        J = np.hstack((J, np.zeros([2,param.nbVarX-move_joint-1]))) # Augmented form # 扩展
        print('np.zeros([2,param.nbVarX-move_joint-1]):', np.zeros([2,param.nbVarX-move_joint-1]) )
        print('J\'', J)
        J = J @ C # Imposed coordination
        print('J:', J)
        # IK
        pinvJ = np.linalg.inv(J.T @ J + np.eye(J.shape[1]) * 1e-1) @ J.T # Damped pseudoinverse
        u = C @ pinvJ @ df # Control commands with imposed coordination

    return 0.1 * u / param.dt   # Velocity in rad/s

NameError: name 'np' is not defined

In [7]:
import numpy as np

a = np.zeros([2,3-2-1])
a
# print(np.zeros([2,3-2-1]))

array([], shape=(2, 0), dtype=float64)

In [ ]:
# Initialization
a = .7
x = np.array([np.pi/2-a, 2*a, -a, 3*np.pi/4, 3*np.pi/4]) # Initial robot state

# Control loop definition
def control(x, param):
    # Prioritized control (CoM tracking prioritized over position tracking)
    u = np.zeros(param.nbVarX)

    if move_joint >= 0:
        # Residuals and Jacobian for primary task # 第一任务
        df1 = (param.Mu_CoM - fkin_CoM(x, param)) * 5

        print('param.Mu_CoM:', param.Mu_CoM)
        print('fkin_CoM(x, param):', fkin_CoM(x, param))

        J1 = Jkin_CoM(x, param)
        df1 = df1[:1] # Track only horizontal location of CoM
        J1 = J1[:1,:] # Track only horizontal location of CoM x轴方向

        # Residual and Jacobian for secondary task # 第二任务
        df2 = (mouse - fkin(x[:move_joint+1], param2)) * 5  # 鼠标点击移动move_joint下标的关节
        # print('df2:', df2)
        print('move_joint:', move_joint)
        J2 = Jkin(x[:move_joint+1], param2)
        # print('J2:', J2)
        J2 = np.hstack((J2, np.zeros([2,param.nbVarX-move_joint-1]))) # Augmented form
        # print('J2:', J2)

        # Prioritized control
        print('J1:', J1)
        print('J2:', J2)
        pinvJ1 = np.linalg.inv(J1.T @ J1 + np.eye(J1.shape[1]) * 1e-1) @ J1.T # Damped pseudoinverse
        N1 = np.eye(param.nbVarX) - pinvJ1 @ J1 # Nullspace projection operator # 零空间投影算子
        u1 = pinvJ1 @ df1 # Command for position tracking
        J2N1 = J2 @ N1
        pinvJ2N1 = np.linalg.inv(J2N1.T @ J2N1 + np.eye(J2N1.shape[1]) * 1e5) @ J2N1.T # Damped pseudoinverse
        u2 = pinvJ2N1 @ (df2 - J2 @ u1) # Command for orientation tracking (with position tracking prioritized)  # 定向跟踪命令（优先考虑位置（质心x方向）跟踪）
        u = u1 + N1 @ u2 # Control commands

    return 0.1 * u / param.dt   # Velocity in rad/s